# Прогноз стоимости автомобилей
Задача: спрогнозировать стоимость автомобилей, используя данные с сайта www.auto.ru

In [ ]:
# Импортируем библиотеки:
import pandas as pd
import numpy as np
import time
import os

from pandas import Series
import re

import pandas_profiling
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

from sklearn.ensemble import StackingRegressor

from sklearn.base import clone
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold

from tqdm import tqdm
from datetime import timedelta, datetime, date

from catboost import CatBoostRegressor
from sklearn.ensemble import (AdaBoostClassifier, GradientBoostingClassifier,
                              RandomForestClassifier, ExtraTreesClassifier)
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

pd.options.mode.chained_assignment = None

In [ ]:
# Функции для обработки данных:
def visualizing_number(column): # визуализация для численных признаков
    bins = 100
    if column.nunique() < 100:
        bins = column.nunique()
    column.hist(bins = bins)
    plt.show()
    sns.boxplot(y = column,data=data)
    plt.show()
    

def get_outliers(column):  # подсчет количества выбросов
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1
    min_out = Q1 - 1.5 * IQR
    max_out = Q3 + 1.5 * IQR
    return (column < min_out).sum() + (column > max_out).sum(), min_out, max_out

# Напишем функцию для подсчёта метрики MAPE:
def mape(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred) / y_test))

# Напишем функцию для отображения метрики MAPE и времени, затраченного на обучение:
def print_learn_report(start, y_test, y_pred):
    print('\nВремя выполнения - ', datetime.now() - start)
    print(f"Точность по метрике MAPE:{(mape(y_test, y_pred))*100:0.2f}%")

# Первичный осмотр данных
В текущем проекте у нас есть только данные для теста, поэтому мы предварительно спарсили данные для трейна отдельно. Проанализируем данные:


In [ ]:
test = pd.read_csv('../input/sf-dst-car-price-prediction/test.csv')
train = pd.read_csv('../input/autoru-parsed-0603-1304/new_data_99_06_03_13_04.csv')
sample_submission = pd.read_csv('../input/sf-dst-car-price-prediction/sample_submission.csv')

In [ ]:
test.sample(3)

In [ ]:
test.info()

In [ ]:
train.sample(3)

In [ ]:
train.info()

In [ ]:
# Целевая переменная - Price. Видим, что в трейне нет пропущенных значений с ценой. 
# Создадим в тесте переменную Price и заполним её Nan.
test['Price'] = np.nan

In [ ]:
# Создадим в трейне sell_id по аналогии с тестом и заполним её Nan:
train['sell_id'] = np.nan

In [ ]:
# Создадим список общих переменных для теста и трейна:
columns = ['sell_id', 'bodyType', 'brand', 'color', 'fuelType', 'modelDate',
           'name', 'numberOfDoors', 'productionDate', 'vehicleConfiguration',
           'engineDisplacement', 'enginePower', 'description', 'mileage',
           'Привод', 'Руль', 'Владельцы', 'ПТС', 'Таможня','Владение', 'Price']

In [ ]:
test.columns

In [ ]:
# Приведём датасеты тест и трейн к единому содержанию переменных:
df_train = train[columns]
df_test = test[columns]

In [ ]:
df_train['sample'] = 1 # помечаем, где у нас трейн
df_test['sample'] = 0 # помечаем, где у нас тест

Проанализируем бренды автомобилей в тесте и трейне - удостоверимся, что они совпадают. 

In [ ]:
df_train.brand.unique()

In [ ]:
df_test.brand.unique()

In [ ]:
# Удалим бренд SUZUKI из нашего трейна:
df_train = df_train.loc[df_train['brand'] != 'SUZUKI']

### Дубликаты
Посмотрим, есть ли дубликаты в трейне и тесте:

In [ ]:
# Видим, что в трейне есть 818 дубликатов:
len(df_train) - len(df_train.drop_duplicates())

In [ ]:
# Удалим их:
df_train = df_train.drop_duplicates()

In [ ]:
# В тесте дубликатов нет:
len(df_test) - len(df_test.drop_duplicates())

In [ ]:
# Заранее обработаем признак "Владельцы" в тесте:
df_test['Владельцы'] = df_test['Владельцы'].str.extract('(\d)', expand=False).str.strip()

In [ ]:
# Объединяем трейн и тест в один датасет, оставив только пересекающиеся признаки.
# Это необходимо для корректной работы с данными в будущем:
data = df_test.append(df_train, sort=False).reset_index(drop=True)

## Примечание 1
При слиянии трейна и теста мы отказались от следующих параметров:
- **car_url, image** - это ссылки, они не предоставляют ценность для нашей модели,
- **complectation_dict, equipment_dict, super_gen, model_info** - это словари данных, сложны в обработке + с большим количеством пропусков,
- **parsing_unixtime** - время парсинга данных, не предоставляет ценность,
- **priceCurrency, Состояние** - перманентные и неизменные величины, не имеют ценности для модели,
- **vendor** - продавец, 2 параметра - европейский и японский (на данном этапе сложно оценить значимость признака),
- **model_name** - модель автомобиля (на данном этапе сложно оценить значимость признака),
- **vehicleTransmission** - трансмиссия (на данном этапе сложно оценить значимость признака).

То есть большинство признаков, от которых мы отказались при слиянии трейна и теста, - незначительны. Логически важным "потерянным" признаком стал столбец с трансмиссией автомобиля. Подумаем позже, учитывать ли его при обучении модели или лучше будет извлечь данные из признака **name** (технические характеристики - мощность двигателя, трансмиссия).

## Примечание 2

Данные для теста были спарсены в промежутке с 19 по 26 октября 2020 года, а данные для трейна мы взяли из датасета на kaggle - они были спарсены с 6 марта по 13 апреля 2020 года. 

Курс доллара в марте / апреле и в октябре 2020 года в среднем составлял около 78 RUB, поэтому делаем предположение, что эконометрические события при создании модели были учтены.

In [ ]:
print(test.parsing_unixtime.min())
print(test.parsing_unixtime.max())

# EDA

Для начала приведём названия всех столбцов к общему типу:

In [ ]:
data.rename(columns={'bodyType': 'body_type', 
                     'engineDisplacement': 'engine_volume',
                     'enginePower': 'engine_power',
                     'fuelType': 'fuel_type',
                     'modelDate': 'model_date',
                     'numberOfDoors': 'number_of_doors',
                     'productionDate': 'production_date',
                     'vehicleConfiguration': 'vehicle_configuration',
                     'Владельцы': 'owners_qty',
                     'Владение': 'ownership_time',
                     'ПТС': 'licence',
                     'Привод': 'type_of_drive',
                     'Руль': 'steering_wheel',
                     'Таможня': 'customs', 
                     'Price': 'price',}, inplace=True)

## Признаки датасета

- body_type - тип кузова,
- brand - марка автомобиля,
- color - цвет автомобиля,
- description - описание в объявлении,
- engine_displacement - объём двигателя,
- engine_power - мощность двигателя,
- fuel_type - тип топлива,
- mileage - пробег,
- model_date - дата релиза модели,
- name - мощность двигателя, трансмиссия,
- number_of_doors - количество дверей, 
- production_date - дата производства автомобиля,
- sell_id - id объявления,
- vehicle_configuration - конфигурация транспортного средства (ТС),
- owners_qty - количество владельцев,
- ownership_time - период владения ТС,
- licence - паспорт ТС,
- type_of_drive - тип привода,
- steering_wheel - сторона руля,
- customs - этап растаможки,
- price - цена автомобиля, целевой параметр,
- sample - индикатор принадлежности данных к тесту (0) и трейну (1).

## Pandas Profiling

Pandas Profiling позволит нам произвести быстрый и углубленный EDA ещё не очищенных данных. После знакомства с отчётом сделаем выводы.

In [ ]:
# pandas_profiling.ProfileReport(data)

## Итак:
1) В датасете представлено 22 признака, из них:
- категориальных переменных - 12,
- числовых переменных - 6,
- неподдерживаемых признаков - 3.

2) Больше всего **пропусков** в переменных:
- sell_id (75.4%) - это логично - в трейне у нас не было этого признака, поэтому мы его добавили сами, 
- description (2.3%),
- ownership_time (64.0%),
- price (24.6%) - это логично - в тесте целевой признак неизвестен.

3) **Высокая вариативность** замечена в следующих категориальных признаках: body_type (132 уникальных значения), name (3723), vehicle_configuration (638), engine_volume (480), ownership_time (564). Для dummy переменных столько значений - это много. Подумаем в будущем над группировкой значений для уменьшения вариативности. 

4) Столбец с **перманентными значениями** customs позже удалим, так как он не имеет ценности для нашей модели.

5) **description** - интересный признак, в будущем посмотрим на него поближе и решим, будем ли извлекать из него информацию с помощью регулярных выражений или нет.

6) **Сильно скоррелированные между собой признаки**:
- model_date / production_date, в будущем решим, будем ли удалять один из этих признаков или нет,
- milage / production_date (model_date),
- **у целевой переменной price** сильная корелляция с признаками mileage и production_date (model_date) - это ценные признаки для будущей модели, подумаем над обработкой дат и созданием новых признаков на их базе (Feature Engineering).




## Обработка признаков

In [ ]:
data.columns

In [ ]:
# Создадим пустые списки категориальных и чиленных признаков. Постепенно будем их заполнять:
cat_cols = []
num_cols = []

## 1. sell_id
Оставляем без изменений

## 2. body_type

In [ ]:
# Посмотрим на уникальные данные признака:
data.body_type.unique()

In [ ]:
# Унифицируем названия кузовов: 
data['body_type'] = data['body_type'].astype(str).apply(lambda x: None if x.strip()=='' else x)
data['body_type'] = data['body_type'].apply(lambda x: x.split(' ')[0].lower())

In [ ]:
data['body_type'].value_counts()

In [ ]:
# Посмотрим на пропуск:
data[data['body_type'] == 'nan']

In [ ]:
# Удалим его:
data = data.loc[data['body_type'] != 'nan']

In [ ]:
data['body_type'].isna().sum()

In [ ]:
# Унифицируем названия кузовов:
body_type_dict = {'внедорожник':'SUV', 
                  'седан':'sedan',
                  'хэтчбек':'hatchback',
                  'лифтбек':'liftback',
                  'универсал':'station_wagon',
                  'минивэн':'minivan',
                  'купе':'coupe',
                  'компактвэн':'compact_van',
                  'пикап':'pickup',
                  'купе-хардтоп':'coupe_hardtop',
                  'фургон':'van',
                  'родстер':'roadster',
                  'кабриолет':'cabriolet',
                  'седан-хардтоп':'sedan_hardtop',
                  'микровэн':'microvan',
                  'лимузин':'limousine',
                  'тарга':'targa', 
                  'фастбек':'fastback'
             }
data['body_type'] = data['body_type'].map(body_type_dict)

In [ ]:
sns.countplot(y = data['body_type'], data = data)

Самые популярные кузовы - внедорожник и седан.

In [ ]:
cat_cols.append('body_type')

## 3. brand

In [ ]:
sns.countplot(y = data['brand'], data = data)

In [ ]:
data['brand'].value_counts()

In [ ]:
cat_cols.append('brand')

## 4. color

In [ ]:
data.color.value_counts()

In [ ]:
# Унифицируем названия цветов:
color_dict = {'040001':'black', 
              'FAFBFB':'white',
              'CACECB':'silver',
              '97948F':'grey',
              'чёрный':'black',
              '0000CC':'blue',
              'белый':'white',
              '200204':'brown',
              'EE1D19':'red',
              'серебристый':'silver',
              'серый':'grey',
              'синий':'blue',
              '007F00':'green',
              'C49648':'beige',
              'красный':'red', 
              'коричневый':'brown',
              '22A0F8':'light_blue',
              'зелёный':'green',
              '660099':'purple',
              'DEA522':'gold',
              '4A2197':'violet',
              'бежевый':'beige',
              'FFD600':'yellow',
              'голубой':'light_blue',
              'FF8649':'orange',
              'золотистый':'gold',
              'пурпурный':'purple',
              'фиолетовый':'violet',
              'жёлтый':'yellow',
              'оранжевый':'orange',
              'FFC0CB':'pink',
              'розовый':'pink'
             }
data['color'] = data['color'].map(color_dict)

In [ ]:
sns.countplot(y = data['color'], data = data)

Самый популярные цвета среди автомобилей - чёрный и белый, а самый редкий - розовый.

In [ ]:
cat_cols.append('color')

## 5. fuel_type

In [ ]:
data['fuel_type'].value_counts()

In [ ]:
# Унифицируем названия топлива:
fuel_type_dict = {'бензин':'petrol', 
                  'дизель':'diesel',
                  'гибрид':'hybrid',
                  'электро':'electro',
                  'газ':'gas'
             }
data['fuel_type'] = data['fuel_type'].map(fuel_type_dict)

In [ ]:
sns.countplot(y = data['fuel_type'], data = data)

Гибридные, газовые и электромобили в самом начале развития своего спроса. 

In [ ]:
cat_cols.append('fuel_type')

## 6. model_date / production_date

In [ ]:
data['model_date'].value_counts()

In [ ]:
data['production_date'].value_counts()

In [ ]:
# Приведём данные с годами к более удобному формату - вычислим возраст автомобиля:
data['model_date'] = data['model_date'].astype('int')

data['model_date'] = 2020 - data['model_date'] # возраст модели 
data['production_date'] = 2020 - data['production_date'] # возраст самого автомобиля

In [ ]:
# Посмотрим на распределение признака model_date:
plt.figure(figsize=(16,6))
visualizing_number(data['model_date'])

In [ ]:
# Проверим наличие выбросов по интеквартильному методу для model_date:
get_outliers(data['model_date'])

Распределение model_date неравномерное и смещено влево. По интеквартильному методу в признаке 6973 выброса, а границы [-2.5, 25.5]. Рассуждаем здраво и понимаем, что минимальный возраст для автомобиля - 0 лет, а максимальный может превышать 25.5 лет. Предполагаем, что выбросов нет.

In [ ]:
# Посмотрим на распределение признака production_date:
plt.figure(figsize=(16,6))
visualizing_number(data['production_date'])

In [ ]:
# Проверим наличие выбросов по интеквартильному методу для model_date:
get_outliers(data['production_date'])

Распределение production_date неравномерное и смещено влево. По интеквартильному методу в признаке 6085 выброса, а границы [-4.5, 23.5]. Рассуждаем здраво и понимаем, что минимальный возраст для автомобиля - 0 лет, а максимальный может превышать 23.5 лет. Предполагаем, что выбросов нет.

In [ ]:
# Скорректируем названия признаков:
data.rename(columns={'model_date': 'model_age', 
                     'production_date': 'car_age'
                    }, inplace=True)

In [ ]:
num_cols.append('model_age')
num_cols.append('car_age')

## 7. name (transmission)

In [ ]:
# Посмотрим на признак:
data['name']

Разберём формат записи: 2.4d AT (200 л.с.) 4WD:
- 2.4d - объём двигателя, у нас есть отдельный признак **engine_volume**,
- AT - **тип трансмиссии**, как упомянули выше, этот признак логически ценен, будем извлекать его,
- 200 л.с. - мощность двигателя, у нас есть отдельный признак **engine_power**,
- 4WD - обозначает полный привод трансмиссии, у нас есть отдельный признак **type_of_drive**.

In [ ]:
# Приведём признак name к строковому типу:
data['name'] = data['name'].astype(str)

In [ ]:
# Создадим новый признак transmission на базе данных из name (используем regex):
data['transmission'] = data['name'].str.extract('([A][T]|[M][T]|[A][M][T]|[C][V][T])',
                                                expand=False).str.strip()

In [ ]:
data['transmission'].value_counts()

Типы трансмиссий:
- AT - автоматическая коробка передач,
- MT - механическая коробка передач,
- AMT - автоматическая ручная коробка передач,
- CVT - бесступенчатая трансмиссия.

In [ ]:
sns.countplot(x = data['transmission'], data = data)

In [ ]:
cat_cols.append('transmission')

In [ ]:
# Удалим признак name, он нам больше не понадобится:
data = data.drop('name', 1)

## 8. number_of_doors

In [ ]:
# Приведём признак к числовому формату:
data['number_of_doors'] = data['number_of_doors'].astype(int)

In [ ]:
# Посмотрим на значения признака:
data['number_of_doors'].value_counts()

In [ ]:
# Есть предположение, что у автомобиля не может НЕ быть дверей. Посмотрим на него:
data[data['number_of_doors'] == 0]

После поиска в Google, понимаем, что это антикварный автомобиль и у него, действительно, нет дверей. Оставляем данные неизменными.

In [ ]:
visualizing_number(data['number_of_doors'])

In [ ]:
cat_cols.append('number_of_doors')

## 9. vehicle_configuration

In [ ]:
data['vehicle_configuration'].value_counts()

In [ ]:
data['vehicle_configuration'] = data['vehicle_configuration'].astype(
                str).apply(lambda x: x if len(x) == 1 else x.split())

In [ ]:
# Извлечём из данных ключевые слова, определяющие конфигурацию автомобиля:
data['vehicle_configuration'] = data['vehicle_configuration'].apply(
    lambda x: x[0].lower() if len(x) == 1 else x[1].lower())

In [ ]:
sns.countplot(x = data['vehicle_configuration'], data = data) 

In [ ]:
cat_cols.append('vehicle_configuration')

## 10. engine_volume

In [ ]:
data['engine_volume']

In [ ]:
# Приведём признак engine_volume к строковому типу:
data['engine_volume'] = data['engine_volume'].astype(str)

In [ ]:
# Извлечём из признака только числовые данные (объём двигателя в литрах) с помощью regex:
data['engine_volume'] = data['engine_volume'].str.extract('(\d.\d)',expand=False).str.strip()

In [ ]:
# Видим, что объём двигателя не везде представлен корректно:
data['engine_volume'].value_counts()

In [ ]:
# Упорядочим распределение признака:
engine_dict = {'1.0': '1.0', 
              '1.1':'1.1', 
              '1.2':'1.2', 
              '1.3':'1.3',
              '1.4':'1.4',
              '1.5':'1.5',
              '1.6':'1.6',
              '1.7':'1.7',
              '1.8':'1.8',
              '1.9':'1.9',
              '2.0':'2.0',
              '2.1':'2.1',
              '2.2':'2.2',
              '2.3':'2.3',
              '2.4':'2.4',
              '2.5':'2.5',
              '2.6':'2.6',
              '2.7':'2.7',
              '2.8':'2.8',
              '2.9':'2.9',
              '3.0':'3.0',
              '3.1':'3.1',
              '3.2':'3.2',
              '3.3':'3.3',
              '3.4':'3.4',
              '3.5':'3.5',
              '3.6':'3.6',
              '3.7':'3.7',
              '3.8':'3.8',
              '3.9':'3.9',
              '4.0':'4.0',
              '4.1':'4.1',
              '4.2':'4.2',
              '4.3':'4.3',
              '4.4':'4.4',
              '4.5':'4.5',
              '4.6':'4.6',
              '4.7':'4.7',
              '4.8':'4.8',
              '4.9':'4.9',
              '5.0':'5.0',
              '5.1':'5.1',
              '5.2':'5.2',
              '5.3':'5.3',
              '5.4':'5.4',
              '5.5':'5.5',
              '5.6':'5.6',
              '5.7':'5.7',
              '5.8':'5.8',
              '5.9':'5.9',
              '6.0':'6.0',
              '6.1':'6.1',
              '6.2':'6.2',
              '6.3':'6.3',
              '6.4':'6.4',
              '6.5':'6.5',
              '6.6':'6.6',
              '6.7':'6.7',
              '6.8':'6.8',
              '6.9':'6.9',
              '7.0':'7.0'
             }
data['engine_volume'] = data['engine_volume'].map(engine_dict)

In [ ]:
# Посмотрим на распределение:
plt.figure(figsize=(16,16))
sns.countplot(y = data['engine_volume'], data = data) 

In [ ]:
# Среднее значение объёма двигателя в датасете - 2.27 литра:
data['engine_volume'].astype(float).describe()

In [ ]:
# Заменим все пропуски на среднее значение признака:
data['engine_volume'] = data['engine_volume'].fillna('2.2')

In [ ]:
data['engine_volume'].value_counts()

In [ ]:
cat_cols.append('engine_volume')

## 11. engine_power

In [ ]:
# Посмотрим на уникальные значения признака:
data['engine_power'].unique()

In [ ]:
# Приведём данные к общему числовому виду:
data['engine_power'] = data['engine_power'].astype(str).apply(lambda x: x.split()[0])
data['engine_power'] = data['engine_power'].astype(float)

In [ ]:
data['engine_power'].value_counts()

In [ ]:
# Минимальное значение мощности двигателя - 19 л.с., максимальное - 639 л.с:
data['engine_power'].describe()

In [ ]:
# Разобьём признак на категории, для этого напишем функцию:
def engine_power(x):
    if x < 100: x = 1
    elif 99 < x < 150: x = 2
    elif 149 < x < 200: x = 3
    elif 199 < x < 250: x = 4
    elif 249 < x < 300: x = 5
    elif 299 < x < 350: x = 6
    elif 349 < x < 400: x = 7
    elif 399 < x < 450: x = 8
    elif 449 < x < 500: x = 9
    elif 499 < x < 550: x = 10
    elif 549 < x < 600: x = 11
    else: x = 12
    return x  

In [ ]:
data['engine_power'] = data['engine_power'].map(engine_power)

In [ ]:
# Распределение мощностей двигателей:
plt.figure(figsize=(16,8))
sns.countplot(x = data['engine_power'], data = data) 

Больше всего автомобилей с мощностью двигателя от 99 до 200 лошадиных сил.

In [ ]:
cat_cols.append('engine_power')

## 12. description

In [ ]:
# Посмотрим на содержание нескольких объявлений:
data['description'].unique()

В описаниях к объявлениям находится много разрозненной информации. Гипотетически можно было бы отсортировать их по ключевым словам, выявив лояльных продавцов. На данном этапе удалим признак desciption. Возможно, во время Feature Engeneering вернёмся к нему снова. 

In [ ]:
data = data.drop('description', 1)

## 13. mileage

In [ ]:
data['mileage'].value_counts()

In [ ]:
# Удостоверимся, что нет пропусков:
data['mileage'].isna().sum()

In [ ]:
# Посмотрим на распределение признака:
visualizing_number(data['mileage'])

Распределение смещено влево. Возможно, в будущем понадобится логарифмирование для улучшения распределения.

In [ ]:
# Посмотрим на выбросы:
get_outliers(data['mileage'])

In [ ]:
data['mileage'].describe()

Отрицательных значений в признаке нет. При этом по интерквартильному методу было определено 3872 выброса. Максимальный выброс - 1000000 км. 

In [ ]:
# Посмотрим на автомобили с пробегом 1000000км:
data.loc[data['mileage'] == 1000000]

Логически рассуждая, предполагаем, что автомобили с возрастом от 11 до 42 лет вполне могли проехать столько км. Считаем, что выбросов нет.

In [ ]:
# Посмотрим распределение признака после логарифмирования,оно сместилось сильно вправо.
# Оставим признак без изменений:
visualizing_number(np.log(data['mileage'] + 1))

In [ ]:
num_cols.append('mileage')

## 14. type_of_drive

In [ ]:
data['type_of_drive'].value_counts()

In [ ]:
# Унифицируем названия приводов:
type_of_drive_dict = {'полный':'four_wheel', 
                      'передний':'front_wheel',
                      'задний':'rear'
                     }
data['type_of_drive'] = data['type_of_drive'].map(type_of_drive_dict)

In [ ]:
sns.countplot(x = data['type_of_drive'], data = data)

В признаке "тип привода" нет пропусков и все данные выглядят корректно.

In [ ]:
cat_cols.append('type_of_drive')

## 15. steering_wheel

In [ ]:
data['steering_wheel'].value_counts()

In [ ]:
# Приведём признак с позицией руля к единообразию:
steering_wheel_dict = {'LEFT':'left', 
                       'Левый':'left',
                       'RIGHT':'right', 
                       'Правый':'right',
                        }
data['steering_wheel'] = data['steering_wheel'].map(steering_wheel_dict)

In [ ]:
data['steering_wheel'].value_counts()

У нас появился первый бинарный признак.

In [ ]:
bin_cols = []
bin_cols.append('steering_wheel')

## 16. owners_qty

In [ ]:
data['owners_qty'].value_counts()

In [ ]:
# Заполним пропуски на самое популярное значение:
data['owners_qty'] = data['owners_qty'].fillna('3')

In [ ]:
# Преобразуем типы данных str и float в int:
for i in data['owners_qty']:
    data['owners_qty'] = data['owners_qty'].astype('int')

In [ ]:
sns.countplot(x = data['owners_qty'], data = data)

Итак, признак **owners_qty** означает:
- 1 - у автомобиля был 1 владелец,
- 2 - было 2 владельца,
- 3 - было 3 и более владельцев.

In [ ]:
cat_cols.append('owners_qty')

## 17. licence

In [ ]:
data['licence'].value_counts()

In [ ]:
# Стандартизируем названия:
licence_dict = {'ORIGINAL':'original', 
                'Оригинал':'original', 
                'DUPLICATE':'duplicate', 
                'Дубликат':'duplicate'
             }
data['licence'] = data['licence'].map(licence_dict)

In [ ]:
# В итоге получаем 3 пропуска:
data['licence'].isna().sum()

In [ ]:
# Заменяем пропуски на самое популярное значение - original:
data['licence'] = data['licence'].fillna('original')

In [ ]:
sns.countplot(x = data['licence'], data = data)

In [ ]:
bin_cols.append('licence')

## 18. customs

In [ ]:
data['customs'].unique()

In [ ]:
# Удаляем признак:
data = data.drop('customs', 1)

## 19. ownership_time

In [ ]:
data['ownership_time'].value_counts()

In [ ]:
# Посмотрим на количество пропусков в признаке:
data['ownership_time'].isna().sum()

In [ ]:
((data['ownership_time'].isna().sum()) / len(data)) * 100

64% пропусков - это критичный показатель. Удалим признак из датасета:

In [ ]:
data = data.drop('ownership_time', 1)

In [ ]:
# Посмотрим на то, как выглядит датасет сейчас:
data.sample(5)

In [ ]:
# Мы привели датасет к единообразному виду. Посмотрим, есть ли в нём ещё дубликаты:
len(data) - len(data.drop_duplicates())

In [ ]:
# Да, было найдено 3981 дубликат. Удалим их:
data = data.drop_duplicates()

In [ ]:
# Удостоверимся,что в рассматриваемом датасете не осталось пропусков:
data.isna().sum()

In [ ]:
data1 = data.copy()

# Анализ признаков по типам

In [ ]:
print(bin_cols)
print(num_cols)
print(cat_cols)

## Анализ бинарных признаков

In [ ]:
# Заменим значения бинарных признаков на 0 и 1:
label_encoder = LabelEncoder()
for i in bin_cols:
    data[i] = label_encoder.fit_transform(data[i])
    
data[bin_cols].head(5)

Посмотрим значимость бинарных признаков:

In [ ]:
imp_cat = Series(mutual_info_classif(data[data['price'].isna() == False][bin_cols], 
                                     data[data['price'].isna() == False]['price'],
                                     discrete_features=True), index=bin_cols)
imp_cat.sort_values(inplace=True)
imp_cat.plot(kind='barh')

## Анализ численных признаков

In [ ]:
plt.figure(figsize=(10,5))
sns.heatmap(data[num_cols+['price']].corr(), annot=True)

На цену автомобиля в равной степени влияют 3 признака: пробег, возраст машины и возраст модели. Между собой сильно скореллированы параметры car_age / model_age (0.97!), а также mileage / model_age и mileage / car_age.

In [ ]:
# Удалим из датасета параметр model_age, при этом оставим идентичный признак car_age:
data = data.drop('model_age', 1)
num_cols.remove('model_age')

In [ ]:
# Посмотрим на значимость непрерывных переменных:
imp_num = Series(f_classif(data[data['price'].isna() == False][num_cols], 
                           data[data['price'].isna() == False]['price'])[0], 
                           index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

## Анализ категориальных признаков

In [ ]:
# Преобразуем все значения категориальных признаков в числа:
for i in cat_cols:
    label_encoder.fit(data[i])
    data[i] = label_encoder.transform(data[i])

In [ ]:
# Посмотрим на значимость категориальных признаков:
imp_cat = Series(mutual_info_classif(data[data['price'].isna() == False][cat_cols], 
                                     data[data['price'].isna() == False]['price'],
                                     discrete_features = True), index = cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

Интересные наблюдения:
- в тройку самых значимых категориальных признаков вошли: объём и мощность двигателя, а также бренд автомобиля,
- цвет авто влияет на выбор покупателей в большей степени, чем типы двигателя или трансмиссии.

In [ ]:
# Посмотрим значимость всех переменных на одном графике:
imp_cat = Series(mutual_info_classif(data[data['price'].isna() == False][cat_cols + num_cols + bin_cols], 
                                     data[data['price'].isna() == False]['price'],
                                     discrete_features = True), index = cat_cols + num_cols + bin_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

## Подытог EDA

Мы обработали все признаки:
- определили типы переменных,
- извлекли признаки из существующих данных,
- заполнили пропуски,
- проанилизировали выбросы,
- стандартизировали общий вид данных,
- проанилизировали взаимосвязь признаков между собой и их с таргетом,
- проанализировали значимость всех признаков для цены автомобиля,
- удалили наименее ценные параметры из датасета.

# Feature Engeneering

## mileage_year

Образуем новый признак mileage_year, который покажет средний пробег автомобиля в год.

In [ ]:
# Добавим на время искусственный признак car_age1 и заменим в нём 0 на 1.
# Это необходимо для того, чтобы в будущем при делении на car_age не было ошибки:
data['car_age1'] = data['car_age']
data.loc[(data['car_age1'] == 0)] = 1

In [ ]:
data['mileage_year'] = round(data['mileage'] / data['car_age']).astype(int)

In [ ]:
 # Удалим вспомогательный столбец:
data = data.drop('car_age1', 1)

In [ ]:
num_cols.append('mileage_year')

In [ ]:
# Посмотрим значимость всех переменных на одном графике:
imp_cat = Series(mutual_info_classif(data[data['price'].isna() == False][cat_cols + num_cols + bin_cols], 
                                     data[data['price'].isna() == False]['price'],
                                     discrete_features = True), index = cat_cols + num_cols + bin_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

# ML

In [ ]:
# Зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
# Фиксируем RANDOM_SEED, чтобы наши эксперименты были воспроизводимы:
RANDOM_SEED = 42
VAL_SIZE = 0.2

## Train test split

In [ ]:
X = data.query('sample == 1').drop(['sell_id', 'sample','price'], axis=1).values
X_sub = data.query('sample == 0').drop(['sample','price'], axis=1).values

In [ ]:
y = data.query('sample == 1')['price'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, 
                                                    shuffle=True, random_state=RANDOM_SEED)

## Model 1: Naive

In [ ]:
start = datetime.now()

naive = LinearRegression().fit(X_train, y_train)
y_pred = naive.predict(X_test)

In [ ]:
print_learn_report(start, y_test, y_pred)

## Model 2: CatBoost

In [ ]:
cb = CatBoostRegressor(iterations = 5000,
                       random_seed = RANDOM_SEED,
                       eval_metric='MAPE',
                       custom_metric=['R2', 'MAE'],
                       silent=True)

In [ ]:
start = datetime.now()

cb.fit(X_train, y_train,
             eval_set=(X_test, y_test),
             verbose_eval=0,
             use_best_model=True)

y_pred = cb.predict(X_test)

cb.save_model('catboost_single_model_baseline.model')

print_learn_report(start, y_test, y_pred)

## Label Encoding

In [ ]:
# Переведём категориальные признаки в dummy переменные:
data1 = pd.get_dummies(data1, columns=cat_cols)

In [ ]:
# Заменим значения бинарных признаков на 0 и 1:
label_encoder = LabelEncoder()
for i in bin_cols:
    data1[i] = label_encoder.fit_transform(data1[i])

In [ ]:
X = data1.query('sample == 1').drop(['sell_id', 'sample','price'], axis=1).values
X_sub = data1.query('sample == 0').drop(['sell_id', 'sample','price'], axis=1).values

In [ ]:
y = data1.query('sample == 1')['price'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, 
                                                    shuffle=True, random_state=RANDOM_SEED)

## Model 3: GradientBoosting

In [ ]:
# Расcчитаем MAPE после преобразования категориальынх признаков методом One-Hot-Encoding.
gb = GradientBoostingRegressor(min_samples_split=2, 
                               learning_rate=0.03, 
                               max_depth=10, 
                               n_estimators=1000)

In [ ]:
start = datetime.now()

gb.fit(X_train, y_train)

y_pred = gb.predict(X_test)

print_learn_report(start, y_test, y_pred)

Метрика стала ещё лучше!

## Model 4: LinerRegression

In [ ]:
start = datetime.now()

lin_reg = LinearRegression().fit(X_train, y_train)
y_pred = lin_reg.predict(X_test)

print_learn_report(start, y_test, y_pred)

Линейная регрессия после One-Hot-Encoding тоже показала лучшую метрику MAPE.

## Model 5: RandomForest

In [ ]:
rf = RandomForestRegressor(n_estimators=1000, 
                            n_jobs=-1, 
                            max_depth=15, 
                            max_features='log2', 
                            random_state=RANDOM_SEED, 
                            oob_score=True)  

In [ ]:
start = datetime.now()

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

print_learn_report(start, y_test, y_pred)

На данном этапе мы получили **MAPE 13.57%** - лучший результат показал **GradientBoosting**.

# Stacking. Ansambles of models

In [ ]:
start = datetime.now()

estimators = [('cb', CatBoostRegressor(iterations=5000,
                                       random_seed=RANDOM_SEED,
                                       eval_metric='MAPE',
                                       custom_metric=['R2', 'MAE'],
                                       silent=True)),
              ('rf', RandomForestRegressor(n_estimators=1000,
                                           n_jobs=-1,
                                           max_depth=15,
                                           max_features='log2',
                                           random_state=RANDOM_SEED,
                                           oob_score=True))]


st_ensemble = StackingRegressor(estimators=estimators,
                                final_estimator=GradientBoostingRegressor(
                                min_samples_split=2,
                                learning_rate=0.03,
                                max_depth=10,
                                n_estimators=1000))


st_ensemble.fit(X_train, np.log(y_train))

predict_ensemble = np.exp(st_ensemble.predict(X_test))
print_learn_report(start, y_test, predict_ensemble)

Итак, лучший результат точности показад стекинг на базе GradientBoostingRegressor c CatBoostingRegressor и RandomForest. Финальный результат - 13.45%

# Submission

In [ ]:
st_ensemble.fit(X, np.log(y))

In [ ]:
predict_submission = np.round(np.exp(st_ensemble.predict(X_sub)),-3).astype('int')
predict_submission 

In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_2_v4.csv', index=False)
sample_submission.head(10)

# What's next?
Или что еще можно сделать, чтоб улучшить результат:

* Спарсить свежие данные 
* Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
* Сгенерировать новые признаки
* Попробовать подобрать параметры модели
* Попробовать другие алгоритмы и библиотеки ML
* Сделать Ансамбль моделей, Blending, Stacking